# Iterative phase estimation algorithm 

The iterative phase estimation algorithm is a quantum algorithm to estimate the phase (or eigenvalue) of an eigenvector of a unitary operator [1]. One of the main applications of the algorithm is to estimate eigenvalues (energies of corresponding eigenstates) for some molecule's $H$ Hamiltonian. Because $H$ is a Hermitian operator, not unitary, (the algorithm works only with unitary operators) we can't estimate directly its eigenvalues. Instead, we create some unitary operator from $H$ and estimate not the eigenvalues of $H$, but something different (the **phase**). The good news is that from this **phase** (we will talk about it later) one can obtain the corresponding eigenvalue of $H$. So, in the end, we are not only estimating the **phase** but, what is more important, the desired eigenvalue. Here are the main steps of the algorithm:

1) Create unitary operator $U$ from given $H$: 
$$U = e^{iHt}.$$ 
Here $t$ is a parameter that we can change
   
2) Apply $U$ on the eigenstate $| \psi_k \rangle$ of $H$. For simplicity, we will assume that the eigenstate is given: 

$$U | \psi_k \rangle = e^{iHt} | \psi_k \rangle$$ 

From the Taylor series for the exponent we have:

$$e^{iHt} | \psi_k \rangle = i t H | \psi_k \rangle - \frac{t^2}{2} H^2 | \psi_k \rangle + \frac{t^4}{4} H^4 | \psi_k \rangle - ...$$

And by using the Schrödinger equation $H | \psi_k \rangle = E_{k} | \psi_k \rangle$, we will obtain: 

$$e^{iHt} |\psi_k \rangle = e^{iE_{k}t} |\psi_k \rangle,$$ 

where $E_k$ is the eigenvalue of the corresponding $| \psi_k \rangle$ eigenstate. By the way, the goal of the algorithm is to find $E_k$.

3) It easy to see that:

$$e^{iHt}|\psi_k \rangle = e^{iE_{k}t} |\psi_k \rangle  = e^{i 2 \pi \varphi} |\psi_k \rangle ,$$

where $\varphi = E_{k}t / 2 \pi$ is the **phase** that the algorithm is capable to estimate. After estimating the phase it will be easy to estimate the corresponding eigenvalue:

$$ E_{k} = 2 \pi \varphi / t$$

Now let's see how the algorithm actually works. Firstly, we should import all the packages that we will use.

In [1]:
import numpy as np
from qiskit import *
from qiskit.aqua.operators import WeightedPauliOperator
#from qiskit.aqua.utils.controlled_circuit import get_controlled_circuit
# get_controlled_circuit in Qiskit 0.13.0 stabile version has a bug. 
# Although in the master branch the bug is fixed. So you can copy
# from qiskit this file 
# https://github.com/Qiskit/qiskit-aqua/blob/master/qiskit/aqua/utils/controlled_circuit.py
# and past to the current jupyter notebook's folder. Then uncomment the following line
from controlled_circuit import get_controlled_circuit

Then we are choosing the quantum computer simulator that we will use (namely 'qasm_simulator'). We are using two quantum registers that each has one qubit. $q$ register is our main register where we should encode $| \psi_k \rangle$ eigenstate and work with it. $a$ register contains one ancillary qubit, where the **phase** will be stored. After running the circuit we will measure only ancillary qubit and will **estimate** a piece (a bit) of the **phase** at each **iteration** of the algorithm. The measurement result will be stored in $c$ classical register.

In [2]:
backend = BasicAer.get_backend('qasm_simulator')
q = QuantumRegister(1)
a = QuantumRegister(1)
c = ClassicalRegister(1)

Here we create a method that returns some simple Hamiltonian. By simple we mean it is a diagonal matrix. It will simplify our problem because in that case, the eigenstates coincide with the computational basis ($|0 \rangle$ and $|1 \rangle$). So initialization of eigenstate will be easy. In our tutorial, we will use $|1 \rangle$ eigenstate and we will estimate its eigenvalue.

In [3]:
def simple_hamiltonian_operator(E_1, E_2):
    """
    Creates 0.5*(E_1 + E_2)*I + 0.5*(E_1 - E_2)*Z pauli sum 
    that will be our "simple" Hamiltonian operator. The corresponding 
    matrix for Hamiltonian is [[E_1, 0], [0, E_2]].
    
    """
    pauli_dict = {
        'paulis': [{"coeff": {"imag": 0.0, "real": 0.5 * (E_1 - E_2)}, "label": "Z"},
                   {"coeff": {"imag": 0.0, "real": 0.5 * (E_1 + E_2)}, "label": "I"}
                   ]
    }
    return WeightedPauliOperator.from_dict(pauli_dict)

In this example we set the eigenvalue of $|1 \rangle$ eigenstate equal to **E_2**. Thus the algorithm should estimate the value of **E_2**. The Hamiltonian in matrix form looks like this:

$$H = \begin{pmatrix}
E_1 & 0\\
0 & E_2
\end{pmatrix} $$

This matrix should be exponentiated and applied on $|1 \rangle$ eigenstate:

$$
e^{iHt}  |1 \rangle = \begin{pmatrix}
e^{i E_1 t} & 0\\
0 & e^{i E_2 t}
\end{pmatrix} 
\begin{pmatrix}
0\\
1
\end{pmatrix}= e^{i E_1 t}
\begin{pmatrix}
0\\
1
\end{pmatrix}$$


In [4]:
E_1 = 0
E_2 = 0.656841

H = simple_hamiltonian_operator(E_1, E_2)

The following code creates a circuit for $U = exp(-iHt)$. It is important to choose parameter $t$ for which $\varphi = E_k t / 2 \pi < 1$. If, for example $\varphi = 1.1$, then the $2 \pi \varphi = 2 \pi + 2 \pi \cdot 0.1$ and we will estimate for the phase only $0.1$ value and we will loose some information about the **phase**. Consequently we will estimate wrong value for the energy $E_{k} = 2 \pi \varphi / t$. 

Note, that the **evolve** method adds, in some sense undesired **"-"** sign. Because of the **"-"** sign we will change something in the algorithm. Remember that $\varphi$ is in the exponent $exp(i 2 \pi \varphi)$? In the case of **evolve** method we will have  $exp(-i 2 \pi \varphi) = exp(i 2 \pi (1 - \varphi)) = exp(i 2 \pi \tilde{\varphi})$. Thus, the algorithm will estimate not $\varphi = 1 - \tilde{\varphi}$, but $\tilde{\varphi}$. For simplicity let's omit the tilde above the $\tilde{\varphi}$, but in the end we should remamber to estimate the energy with following equation $E_{k} = 2 \pi (1 - \varphi) / t$.

In [5]:
t = 1
H_circuit = H.evolve(evo_time=t, quantum_registers=q)

Here comes the ancillary qubit. The following code creates a controlled version of the **H_circuit** on $q$ register. The control qubit is the ancillary qubit. This means that if the ancillary is in $|0 \rangle$ state the **H_circuit** will not run on $q$ register and if the ancillary is in $|1 \rangle$ state the **H_circuit** will run on $q$ register. Also, if the ancillary qubit is in the $|+ \rangle$ superposition state and the qubit in $q$ register is in $|1 \rangle$ eigenstate we will have:

$$|+ \rangle_a |1 \rangle_q = \frac{1}{\sqrt{2}} (|0 \rangle_a + |1 \rangle_a)|1 \rangle_q$$

After **control_H** circuit the state of the qubits will look like this:

$$(|0 \rangle_a + exp(-i 2 \pi \varphi) |1 \rangle_a)|1 \rangle_q$$

This way we can store the value of the **phase** in the **phase** of the ancillary qubit. To give you an idea of how we are going to extract the **phase** let's imagine that the phase is either $0$ or $\frac{1}{2}$. If the phase is $0$ the state of the ancillary qubit will be equal to $|+\rangle_a$ state and if it is $\frac{1}{2}$ the state will be equal to  $|- \rangle_a$. After this we apply to the ancillary qubit the Hadamard gate. In this step, the $|+\rangle_a$ state will transform to the $|0\rangle_a$ state and the $|-\rangle_a$ state will transform to the $|0\rangle_a$ state. So, just by measuring the ancillary qubit one will know is the phase $0$ or $\frac{1}{2}$. This is an important example because our goal is to simplify the problem at each iteration of the algorithm to this particular simple example.

In [6]:
control_H = get_controlled_circuit(H_circuit, a[0])

For general case we will represent $\varphi$ in its fractional binary representation:

$$\varphi = 0.\varphi_0 \varphi_1 \varphi_2 \varphi_3... =  \frac{\varphi_0}{2} + \frac{\varphi_1}{2^2} + \frac{\varphi_2}{2^3} + \frac{\varphi_3}{2^4} + ...$$

where each $\varphi_k$ is a bit (it is equal to $0$ or $1$). Here we should do **iterations**. For each iteration, we are going to estimate one $\varphi_k$ bit at a time. Moreover, we should start estimating the bits starting from the last bit. For example let's find $\varphi = 0.11$ **phase**. Notice, that after **cantrol_H** we will have following exponent $exp(i 2 \pi 0.11)$. If we will apply **cantrol_H** twice we will obtain two exponents $exp(i 2 \pi 0.11) \cdot exp(i 2 \pi 0.11) = exp(i 2 \pi 0.1)$, because $2 \cdot 0.11 = 2 \cdot (\frac{1}{2} + \frac{1}{4}) = (1 + \frac{1}{2}) = 1.1$. This way we are coming to the simple example explained above (\varphi is equal to $0$ or $\frac{1}{2}$). So it will be easy to estimate the second bit from $\varphi = 0.11$. At the second iteration, we are going to estimate the first bit of the $\varphi$. For that we will apply only one **cantrol_H** ($exp(i 2 \pi 0.11)$). Now we remamber value of the second bit $\varphi_1$. It will be nice if we will subtract from the **phase** the $\varphi_1$. We will do it with the phase gate. In qiskit this action corresponds to this gate **u1(-phase_shift, a[0])**, where **phase_shift** contains all estimated bits that we should subtract to come to the simple phase case ($0$ or $\frac{1}{2}$ case).

The following **for** loop iteratively estimates the **phase**.

In [7]:
num_bits_estimate = 10
phase = 0
for k_precision in reversed(range(num_bits_estimate)):
    # Create a Quantum Circuit acting on the q register
    k_circ = QuantumCircuit(q, a, c)

    # (1) |1> eigenstate initialization
    k_circ.x(q[0])

    # (2) Initial Hadamard gate applied on ancillary qubit.
    k_circ.h(a[0])

    # (3) The control Hamiltonian applied on the qubits where control qubit is the ancillary qubit.
    for order in range(2 ** k_precision):
        k_circ += control_H

    # (4) The phase gate and final Hadamard gate on ancillary qubit.
    phase_shift = 2 * np.pi * phase * 2 ** k_precision
    k_circ.u1(-phase_shift, a[0])
    k_circ.h(a[0])

    # (5) Measurement of ancillary qubit (findig the bit)
    k_circ.measure(a[0], c[0])

    # (6) executing on Quantum Computer and finding a bit from the phase
    job = execute(k_circ, backend, shots=1000)
    result = job.result()
    counts = result.get_counts()
    value = int(max(counts, key=counts.get))

    # (7) phase after this iteration
    phase += value / 2 ** (k_precision + 1)

Finally we are estimating the eigenvalue (energy) of the $| 1 \rangle$ eigenstate and printing result alongside the true eigenvalue **E_2**.

In [8]:
eigenvalue = 2 * np.pi * (1 - phase) / t
print("Eigenvalue of Hamiltonian: " + str(E_2))
print("Estimated eigenvalue of Hamiltonian: " + str(eigenvalue))

Eigenvalue of Hamiltonian: 0.656841
Estimated eigenvalue of Hamiltonian: 0.6565437772150544


We omitted one step of the algorithm. That is the Trotter decomposition step. Let's say we have some Hamiltonian $H = h_1 + h_2 + h_3$ we want to exponentiate it to obtain $U = exp(iHt)$ unitary. Actually with with this algorithm (namely when we are constructing a circuit with **evolve** method) we obtain $\tilde{U}) = exp(i h_1 t) \cdot exp(i h_2 t) \cdot exp(i h_3 t)$. The thing is that actually $exp(iHt) \ne  exp(i h_1 t) \cdot exp(i h_2 t) \cdot exp(i h_3 t)$. To obtain more or less valid $U$ operator we should do Trotter decomposition:

$$ U = (exp(i h_1 t/N) \cdot exp(i h_2 t/N) \cdot exp(i h_3 t/N))^N + O(\frac{t^2}{N}),$$

where $N$ is Trotter number [2]. By increasing $N$ it is possible to obtain better representation for $U$ operator. Because the example of the Hamiltonian is simple (2x2 diagonal matrix) we obtained a reasonable result without doing this step.

[1] [Quantum phase estimation algorithm](https://en.wikipedia.org/wiki/Quantum_phase_estimation_algorithm)

[2]  [J.D. Whitfield,  J. Biamonte  and  A. Aspuru-Guzik,
Molecular Physics, "Simulation of electronic structure Hamiltonians using
quantum computers" (2011)](https://www.tandfonline.com/doi/abs/10.1080/00268976.2011.552441)